In [70]:
import xml.etree.ElementTree as ET
import os
import pathlib

from unidecode import unidecode

In [71]:
def gerar_arquivo(arquivo, xml):
    tree = ET.parse(xml)

    root = tree.getroot()

    for child in root.findall('DADOS-GERAIS'):
        arquivo.write("Nome: "+child.attrib["NOME-COMPLETO"]+"\n\n")

    for child in root.findall("PRODUCAO-BIBLIOGRAFICA"):
        string = ""
        for artigos in child.findall("ARTIGOS-PUBLICADOS"):
            for artigo in artigos.findall("ARTIGO-PUBLICADO"):
                for dados_basicos in artigo.findall("DADOS-BASICOS-DO-ARTIGO"):

                    string += "\t<Artigo>\n"

                    attr = dados_basicos.attrib
                    doi = ""

                    for i in attr.keys():
                        field_name = convert_attrib_name_to_string(i)
                        field = convert_attrib_name_to_string(attr[i])
                        if "Home page do trabalho" in field_name:
                            field = field.replace("[", "").replace("]", "")

                            if "doi:" in field:
                                aux = field.split("doi:")
                                field = aux[0]
                                doi = aux[1]
                        
                        if "Doi" in field_name and not field and doi:
                            field = doi
                            
                        string += "\t\t"+field_name+": "+field+"\n" if field else ""


                string += "\n"


                for detalhamento in artigo.findall("DETALHAMENTO-DO-ARTIGO"):
                    
                    attr = detalhamento.attrib
                    
                    for i in attr.keys():
                        field_name = convert_attrib_name_to_string(i)
                        field = convert_attrib_name_to_string(attr[i])
                        string += "\t\t"+field_name+": "+field+"\n" if field else ""


                string += ("\n")


                string += "\t\tAutor(es) do artigo: "

                for autores in artigo.findall("AUTORES"):
                    string += autores.attrib["NOME-COMPLETO-DO-AUTOR"]+", "
                
                string = string.rstrip(", ")

                string += "\n\n"

                for palavras_chave in artigo.findall("PALAVRAS-CHAVE"):
                    keywords = palavras_chave.attrib
                    for i in keywords.keys():
                        field_name = convert_attrib_name_to_string(i)
                        field = convert_attrib_name_to_string(keywords[i])
                        string += "\t\t"+field_name+": "+field+"\n" if field else ""
                
                string += "\n"            

                for areas_conhecimento in artigo.findall("AREAS-DO-CONHECIMENTO"):
                    areas = [elem.tag for elem in areas_conhecimento.iter() if elem is not areas_conhecimento]
                    for i in range(len(areas)):
                        for area in areas_conhecimento.findall(areas[i]):
                            for i in area.attrib.keys():
                                field_name = convert_attrib_name_to_string(i)
                                field = convert_attrib_name_to_string(area.attrib[i])
                                string += "\t\t"+field_name+": "+field+"\n" if field else ""

                    
                string += "\n"

                arquivo.write(string)

def convert_attrib_name_to_string(attrib_name):
    return attrib_name.lower().capitalize().replace("-", " ")

def xml_get_first_name(xml):
    tree = ET.parse(xml)
    root = tree.getroot()
    
    nome = ""

    for child in root.findall('DADOS-GERAIS'):
        nome = child.attrib["NOME-COMPLETO"]
    
    primeiro_nome = nome.split(" ")
    
    return primeiro_nome[0]

In [72]:
diretorio_entrada = "./xmls/"
diretorio_saida = "./saidas/"

if not os.path.exists(diretorio_entrada):
    os.makedirs(diretorio_entrada)

if not os.path.exists(diretorio_saida):
    os.makedirs(diretorio_saida)

arquivo = open(diretorio_saida+"todos_curriculos.txt", 'w')


for filename in os.listdir(diretorio_entrada):
    
    f = os.path.join(diretorio_entrada, filename)

    if os.path.isfile(f):
        
        if(pathlib.Path(f).suffix == '.xml'):

            nome_do_arquivo = diretorio_saida+"curriculo_"+unidecode(xml_get_first_name(""+f).lower())+".txt"
            
            print("Gerando o arquivo: "+nome_do_arquivo.replace(diretorio_saida, ""))

            other_file = open(nome_do_arquivo, "w")

            gerar_arquivo(other_file, ""+f)
            gerar_arquivo(arquivo, ""+f)

            other_file.close()

arquivo.close()


Gerando o arquivo: curriculo_mariana.txt
Gerando o arquivo: curriculo_julia.txt
Gerando o arquivo: curriculo_andre.txt
Gerando o arquivo: curriculo_mirkos.txt
Gerando o arquivo: curriculo_ivana.txt
Gerando o arquivo: curriculo_solange.txt
